### Hyperparameter tuning of Deep learning models in Keras
### Pretrained model with aisaratuner

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend
from keras import applications
import numpy as np
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras import optimizers

In [ ]:
!pip install aisaratuners 
from aisaratuners.aisara_keras_tuner import Hp, HpOptimization

     |████████████████████████████████| 13.2MB 242kB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [ ]:
(img_train, y_train), (img_test, y_test) = keras.datasets.cifar10.load_data()

170500096/170498071 [==============================] - 4s 0us/step


In [ ]:
y_test

array([[3],
       [8],
       [8],
       ...,
       [5],
       [1],
       [7]], dtype=uint8)

In [ ]:
# Normalize pixel values between 0 and 1
x_train = img_train.astype('float32') / 255.0
x_test = img_test.astype('float32') / 255.0

img_rows, img_cols = 32,32
if backend.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 3, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 3, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 3)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 3)
    input_shape = (img_rows, img_cols, 1)

In [ ]:
from keras.utils import to_categorical
y_train=to_categorical(y_train,)
y_test=to_categorical(y_test)

In [ ]:
# define Hps:
hps = Hp()
hp_1 = hps.numrange(name='num_layers',min=2,max=10)
hp_2 = hps.numrange(name='nodes_dense',min=64,max=512)
hp_3 = hps.numrange(name='lr',min=0.0001,max=0.01, type='log')

In [ ]:
# define keras model and return model history

def model_builder(comp, k):
    base_model = applications.VGG16(weights='imagenet', pooling='avg', include_top=False)
    inputs = tf.keras.Input(shape=(32, 32, 3))
    x = base_model(inputs)
    for i in range(comp['num_layers'][k]):
        x = Dense(comp['nodes_dense'][k], activation='relu')(x)
    x = Dense(10, activation='softmax')(x) 
    model = tf.keras.Model(inputs=inputs, outputs=x)
    model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(comp['lr'][k]),metrics=['acc'])
    history = model.fit(x_train,y_train,epochs=5,validation_data=(x_test,y_test))
    return model, history

In [ ]:
# run optimizer
demo = HpOptimization(hps, model_builder, ['val_acc','val_loss'], ['max','min'], num_trials=3, rounds=1, mode='p')
%time demo.run_opti()

For commercial use, you can obtain our API from https://rapidapi.com/aisara-technology-aisara-technology-default/api/aisara-hyperparameter-tuning
If you are a private user, set the mode parameter in HpOptimization class to "p".

Round-1:

  Run-1:
58892288/58889256 [==============================] - 1s 0us/step
Epoch 1/5
1563/1563 [==============================] - 33s 21ms/step - loss: 126.4001 - acc: 0.0979 - val_loss: 2.3030 - val_acc: 0.1000
Epoch 2/5
1563/1563 [==============================] - 32s 21ms/step - loss: 2.3032 - acc: 0.0976 - val_loss: 2.3035 - val_acc: 0.1000
Epoch 3/5
1563/1563 [==============================] - 32s 21ms/step - loss: 2.3032 - acc: 0.0990 - val_loss: 2.3029 - val_acc: 0.1000
Epoch 4/5
1563/1563 [==============================] - 32s 21ms/step - loss: 2.3033 - acc: 0.0983 - val_loss: 2.3033 - val_acc: 0.1000
Epoch 5/5
1563/1563 [==============================] - 33s 21ms/step - loss: 2.3033 - acc: 0.0984 - val_loss: 2.3035 - val_acc: 0.1000

  Run-2:


In [ ]:
demo.best_model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
vgg16 (Functional)           (None, 512)               14714688  
_________________________________________________________________
dense_7 (Dense)              (None, 139)               71307     
_________________________________________________________________
dense_8 (Dense)              (None, 139)               19460     
_________________________________________________________________
dense_9 (Dense)              (None, 139)               19460     
_________________________________________________________________
dense_10 (Dense)             (None, 10)                1400      
Total params: 14,826,315
Trainable params: 14,826,315
Non-trainable params: 0
__________________________________________